# 1. Regresión Logística <a id="8"></a>

### Leer el Conjunto de Datos

El conjunto de datos incluye información sobre:

* Clientes que se fueron en el último mes: la columna se llama `Churn`.
* Servicios a los que se ha suscrito cada cliente: teléfono, líneas múltiples, Internet, seguridad en línea, copia de seguridad en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas.
* Información de la cuenta del cliente: cuánto tiempo ha sido cliente, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales.
* Información demográfica sobre clientes: género, rango de edad y si tienen pareja o personas dependientes.

**ChurnData.csv** es un archivo de datos hipotéticos sobre los esfuerzos de una empresa de telecomunicaciones para reducir la rotación de su base de clientes. Cada caso corresponde a un cliente por separado y registra diversa información demográfica y de uso del servicio. 

Cargar los datos y guardarlos en el dataframe `df4`:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
# ruta de datos y leer los datos

path4='datos/ChurnData.csv'
df4 = pd.read_csv(path4)
df4.head()

Tamaño y forma del conjunto de datos.

In [ ]:
print('Tamaño: ', df4.size)
print('Forma: ', df4.shape)

## Pre-procesamiento de los Datos

Primero se seleccionarán algunas características para el modelado. También se procederá a cambiar el tipo de datos de destino para que sea un número entero, ya que es un requisito del módulo a utilizar de `scikit-learn`.

In [ ]:
df4_fuga = df4[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',   'callcard', 'wireless','churn']].copy()
df4_fuga['churn'] = df4_fuga['churn'].astype(int)
df4_fuga.head()

Tamaño y forma del nuevo conjunto de datos.

In [ ]:
print('Tamaño: ', df4_fuga.size)
print('Forma: ', df4_fuga.shape)

Definir **X4** e **y4** a partir del conjunto de datos.

In [ ]:
X4 = np.asarray(df4_fuga[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip']])
X4[0:5]

In [ ]:
y4 = np.asarray(df4_fuga['churn'])
y4 [0:5]

Además, se procederá a normalizar el conjunto de datos asociado a la variable **X4**:

In [ ]:
X4 = preprocessing.StandardScaler().fit_transform(X4)
X4[0:5]

## Configuración del Modelo

Ahora, se procederá a dividir el conjunto de datos en el conjunto de entrenamiento y el conjunto de prueba en una proporción 80/20 respectivamente.


In [ ]:
X4_entrena, X4_prueba, y4_entrena, y4_prueba = train_test_split(X4, y4, test_size=0.2, random_state=4)
print ('Conjunto de Entrenamiento set:', X4_entrena.shape,  y4_entrena.shape)
print ('Conjunto de Prueba:', X4_prueba.shape,  y4_prueba.shape)

## Modelado

El siguiente paso es construir el modelo usando el módulo `LogisticRegression` del la biblioteca `Scikit-learn`. Esta función implementa la regresión logística y puede usar diferentes optimizadores numéricos para encontrar parámetros, incluidos los solucionadores `newton-cg`, `lbfgs`, `liblinear`, `sag`, `saga`. En una búsqueda en Internet se puede encontrar amplia información sobre los pros y los contras de estos optimizadores.

La versión de `LogisticRegression` en `Scikit-learn` admite la regularización. La regularización es una técnica utilizada para resolver el problema de sobreajuste en los modelos de aprendizaje automático.

El parámetro `C` indica la **fuerza de regularización inversa**, que debe ser un valor flotante positivo. Los valores más pequeños especifican una regularización más fuerte.

Ahora se procederá a importar el módulo `LogisticRegression` y posteriormente se ajustará el modelo con el conjunto de entrenamiento.

In [ ]:
from sklearn.linear_model import LogisticRegression

regr_logi = LogisticRegression(C=0.01, solver='liblinear').fit(X4_entrena,y4_entrena)
regr_logi

## Pronóstico

Una vez entrenado el modelo se puede realizar el pronóstico con el conjunto de datos de prueba.

In [ ]:
y4_hat = regr_logi.predict(X4_prueba)
y4_hat

La función `predict_proba` devuelve estimaciones para todas las clases, ordenadas por la etiqueta de las clases. Así, la primera columna es la probabilidad de la clase 0, $P(Y=0|X)$, y la segunda columna es la probabilidad de la clase 1, $P(Y=1|X)$:

In [ ]:
y4_hat_prob = regr_logi.predict_proba(X4_prueba)
y4_hat_prob

## Evaluación

**Índice de Jaccard**

Se utilizará el **índice jaccard** para evaluar la precisión. Este índice se puede definir como el tamaño de la intersección dividido por el tamaño de la unión de dos conjuntos de etiquetas. Si todo el conjunto de etiquetas pronosticadas para una muestra coincide estrictamente con el verdadero conjunto de etiquetas, entonces la precisión del subconjunto es $1,0$; de lo contrario es $0,0$.

In [ ]:
metrics.jaccard_score(y4_prueba, y4_hat)

**Matiz de Confusión**

Otra forma de ver la precisión del clasificador es utilizar la **matriz de confusión**. Esta es una herramienta útil para medir que tan bueno es un modelo clasificación. En particular, sirve para mostrar de forma explícita cuándo una clase es confundida con otra, lo cual, permite trabajar de forma separada con distintos tipos de error. Esta matriz se presenta siempre en forma de tabla, de manera que en cada columna aparece el número de predicciones de cada clase, mientras que cada fila muestra el número real de instancias de cada clase. Es decir, esta matriz pone en relación las predicciones realizadas por un modelo de clasificación y los resultados correctos que debería haber mostrado. Así puede medirse el desempeño del mismo, determinando qué tipo de errores y de aciertos tiene cada modelo.

A continuación se define una función para graficar una matriz de confusión.

In [ ]:
# Esta función imprime y grafica una matriz de confusión.
# Se puede aplicar una normalización configurando el parámetro `normalize=True`.

import itertools

def grafica_matriz_confusion(matr_conf, clases,
                          normalizar=False,
                          titulo='Matriz de Confusión',
                          cmap=plt.cm.Blues):

    if normalizar:
        matr_conf = matr_conf.astype('float') / matr_conf.sum(axis=1)[:, np.newaxis]
        print("Matriz de Confusión Normalizada.")
    else:
        print('Matriz de Confusión matrix sin normalización')

    print(matr_conf)

    plt.imshow(matr_conf, interpolation='nearest', cmap=cmap)
    plt.title(titulo)
    plt.colorbar()
    tick_marks = np.arange(len(clases))
    plt.xticks(tick_marks, clases, rotation=45)
    plt.yticks(tick_marks, clases)

    formato = '.2f' if normalizar else 'd'
    umbral = matr_conf.max() / 2.
    for i, j in itertools.product(range(matr_conf.shape[0]), range(matr_conf.shape[1])):
        plt.text(j, i, format(matr_conf[i, j], formato),
                 horizontalalignment="center",
                 color="white" if matr_conf[i, j] > umbral else "black")

    plt.tight_layout()
    plt.ylabel('Etiqueta valores Verdaderos')
    plt.xlabel('Etiqueta valores Pronosticados')

In [ ]:
# Calcular matriz de confusión

matriz_confusion = metrics.confusion_matrix(y4_prueba, y4_hat, labels=[1,0])
np.set_printoptions(precision=2)

# Grafica matriz de confusión no normalizada
plt.figure()
grafica_matriz_confusion(matriz_confusion, clases=['churn=1','churn=0'],normalizar= False,  titulo='Matriz de Confusión')

**Análisis de la primera fila**: La primera fila es para clientes cuyo valor real de abandono en el conjunto de prueba es 1 (es decir, abandona la compañía). Como se puede calcular, de 40 clientes, 15 de ellos tienen el valor de abandono en 1. Y de estos 15, el clasificador predijo correctamente 6 de ellos como 1 y predijo erróneamente 9 de ellos como 0.

Esto significa que, para 6 clientes, el valor real de abandono fue 1 en el conjunto de prueba, y el clasificador también los predijo correctamente como 1. Sin embargo, mientras que la etiqueta real de 9 clientes fue 1, el clasificador los predijo como 0, lo cual no es correcto. Esto se puede considerar como un error del modelo para la primera fila.

**Análisis de la segunda fila**: Hay 25 clientes cuyo valor de abandono era 0 (es decir, se queda en la compañía). El clasificador predijo correctamente 24 de ellos como 0 y uno de ellos incorrectamente como 1. Por lo tanto, ha hecho un buen trabajo al predecir los clientes con un valor de abandono de 0. 

Lo bueno de la matriz de confusión es que muestra la capacidad del modelo para predecir correctamente o separar las clases. En el caso específico del clasificador binario, como este ejemplo, podemos interpretar estos números como el recuento de verdaderos positivos (TP), falsos positivos (FP), verdaderos negativos (TN) y falsos negativos (FN).

In [ ]:
print (metrics.classification_report(y4_prueba, y4_hat))

Basado en el recuento de cada sección, se puede calcular la precisión y la sensibilidad de cada etiqueta:

* **Precisión** es una medida de la exactitud siempre que se haya predicho una etiqueta de clase, es decir, se refiere a lo cerca que está el resultado de una predicción del valor verdadero. Se define por: $Precision = TP / (TP + FP)$.

* **Sensibilidad** es la tasa positiva verdadera que representa la habilidad del modelo de detectar los casos relevantes. Se define como: $Recall = TP / (TP + FN)$.

Así, ahora se puede calcular la precisión y la sensibilidad de cada clase.

**Valor de F1:**
El siguiente paso es calcular los valores de F1 para cada etiqueta en función de la precisión y la sensibilidad de esa etiqueta.

El valor de F1 es el promedio armónico de precisión y sensibilidad, donde un valor de F1 alcanza su mejor valor en 1 (precisión y sensibilidad perfectas) y el peor en 0. Es una buena manera de mostrar que un clasificador tiene un buen valor para ambos, sensibilidad y precisión.

De esta manera, se puede decir que la precisión promedio de este clasificador es el promedio de la puntuación F1 para ambas etiquetas, el cual es 0,72 en este caso.

**Pérdida Logística**

La pérdida logística o **log-loss** es una métrica que indica qué tan cerca está la probabilidad de predicción del valor real/verdadero correspondiente (0 o 1 en caso de clasificación binaria). Cuanto más diverja la probabilidad predicha del valor real, mayor será el valor de pérdida logarítmica.

Ahora, se probará realizar una evaluación  **log-loss** para el modelo. En la regresión logística, el resultado puede ser que la probabilidad de abandono de clientes sea sí (o igual a 1). Esta probabilidad es un valor entre 0 y 1. Por lo tanto, **log-loss** mide el rendimiento de un clasificador donde la salida prevista es un valor de probabilidad entre 0 y 1.

Este indicador se utiliza, principalmente para comparar modelos. El modelo que tenga el menor valor de **log-loss** será el mejor evaluado.

In [ ]:
print ("Pérdila logística: %.4f" % metrics.log_loss(y4_prueba, y4_hat_prob))